In [11]:
import pandas as pd
import re
from collections import Counter

In [3]:
filepath = '../data/processed/american_artworks_with_dates.csv'
df = pd.read_csv(filepath)
df.head()

,ID,Title,Type,Date,Topics,ProductionYear
0,ld1-1643381040022-1643381068115-0,Governor for Steam Engines,Smithsonian American Art Museum,1870,Patent model,1870.0
1,ld1-1643381040022-1643381068119-0,Mousetrap,Smithsonian American Art Museum,1870,Patent model,1870.0
2,ld1-1643381040022-1643381068106-0,Electrical Timer Bell,Smithsonian American Art Museum,1860,Patent model,1860.0
3,ld1-1643381040022-1643381068107-0,Water Feed for Boilers,Smithsonian American Art Museum,1880,Patent model,1880.0
4,ld1-1643381040022-1643381068114-1,Windmill,Smithsonian American Art Museum,1880,Patent model,1880.0


In [12]:
filepath = '../data/processed/american_artworks_with_dates.csv'
df = pd.read_csv(filepath)

df["ProductionYear"] = pd.to_numeric(df["ProductionYear"], errors='coerce')

min_year = df["ProductionYear"].min()
max_year = df["ProductionYear"].max()

print(f"Earliest Year: {min_year}")
print(f"Latest Year: {max_year}")


Earliest Year: 1100.0
Latest Year: 2010.0


In [39]:
# Load the dataset
filepath = '../data/processed/american_artworks_with_dates.csv'
df = pd.read_csv(filepath)

df["ProductionYear"] = pd.to_numeric(df["ProductionYear"], errors='coerce')

df = df[(df["ProductionYear"] >= 1920) & (df["ProductionYear"] <= 2010)]

# Define common filler words to exclude
stopwords = set(["and", "or", "for", "the", "of", "in", "on", "with", "to", "a", "an", "by", "at", "as", "from", "untitled", "no", "1", "untitled"])

# Function to clean and extract words from a title
def process_title(title):
    words = re.findall(r'\b\w+\b', title.lower())  
    return [word for word in words if word not in stopwords]  

# Group titles by decade
df["Decade"] = (df["ProductionYear"] // 10) * 10  # Convert year to decade

# Create a dictionary to store word counts for each decade
decade_word_counts = {}

# Process each decade separately
for decade, group in df.groupby("Decade"):
    all_words = []
    for title in group["Title"].dropna():
        all_words.extend(process_title(title))  # Process title and add words
    word_counts = Counter(all_words)  # Count word occurrences
    decade_word_counts[decade] = list(word_counts.keys())  # Store unique words

# Convert to a DataFrame
decade_df = pd.DataFrame(list(decade_word_counts.items()), columns=["Decade", "Unique_Words"])

# Display the result
print(decade_df)

# Save the result to a CSV
output_filepath = "../data/processed/decade_unique_words.csv"
decade_df.to_csv(output_filepath, index=False)

   Decade                                       Unique_Words
0  1920.0  [standing, articulated, figure, kenyon, cox, f...
1  1930.0  [shapes, fear, street, shrine, teasel, belliss...
2  1940.0  [field, 2, seated, nude, girl, sister, ella, f...
3  1950.0  [tangiers, new, england, hostess, madonna, sol...
4  1960.0  [man, three, donkeys, children, swimming, moth...
5  1970.0  [tieback, one, pair, landscape, mattress, make...
6  1980.0  [lady, samuel, mink, two, gentlemen, comb, you...
7  1990.0  [memory, vessel, doll, parts, whaler, s, quilt...
8  2000.0  [timelines, tenements, chatelaine, boy, fancy,...
9  2010.0  [cross, catalogue, number, 1938, 52, series, a...


In [41]:
decade_df.head()

,Decade,Unique_Words
0,1920.0,"[standing, articulated, figure, kenyon, cox, f..."
1,1930.0,"[shapes, fear, street, shrine, teasel, belliss..."
2,1940.0,"[field, 2, seated, nude, girl, sister, ella, f..."
3,1950.0,"[tangiers, new, england, hostess, madonna, sol..."
4,1960.0,"[man, three, donkeys, children, swimming, moth..."


In [45]:
decade_df['Word_Count'] = decade_df['Unique_Words'].apply(len)

print(decade_df)

   Decade                                       Unique_Words  Word_Count
0  1920.0  [standing, articulated, figure, kenyon, cox, f...        2035
1  1930.0  [shapes, fear, street, shrine, teasel, belliss...        1257
2  1940.0  [field, 2, seated, nude, girl, sister, ella, f...         680
3  1950.0  [tangiers, new, england, hostess, madonna, sol...          34
4  1960.0  [man, three, donkeys, children, swimming, moth...          38
5  1970.0  [tieback, one, pair, landscape, mattress, make...          26
6  1980.0  [lady, samuel, mink, two, gentlemen, comb, you...          29
7  1990.0  [memory, vessel, doll, parts, whaler, s, quilt...          28
8  2000.0  [timelines, tenements, chatelaine, boy, fancy,...          20
9  2010.0  [cross, catalogue, number, 1938, 52, series, a...          15


In [52]:
import pandas as pd
import re
from collections import Counter

# Load data
filepath = '../data/processed/art_data.json'
df = pd.read_json(filepath)

# Convert 'objectEndDate' to numeric and handle any errors
df["objectEndDate"] = pd.to_numeric(df["objectEndDate"], errors='coerce')

# Filter data for the specified range of years
df = df[(df["objectEndDate"] >= 1920) & (df["objectEndDate"] <= 2035)]

# Define common filler words to exclude
stopwords = set(["and", "or", "for", "the", "of", "in", "on", "with", "to", "a", "an", "by", "at", "as", "from", "untitled", "no", "1"])

# Function to clean and extract words from the title
def process_title(objectName):
    words = re.findall(r'\b\w+\b', objectName.lower())  # Use objectName to extract words
    return [word for word in words if word not in stopwords]  # Exclude stopwords

# Group titles by decade
df["Decade"] = (df["objectEndDate"] // 10) * 10  # Convert year to decade

# Create a dictionary to store word counts for each decade
decade_word_counts = {}

# Process each decade separately
for decade, group in df.groupby("Decade"):
    all_words = []
    for objectName in group["objectName"].dropna():  # Process objectName for each row
        all_words.extend(process_title(objectName))  # Process title and add words
    word_counts = Counter(all_words)  # Count word occurrences
    decade_word_counts[decade] = list(word_counts.keys())  # Store unique words

# Convert to a DataFrame
decade_df = pd.DataFrame(list(decade_word_counts.items()), columns=["Decade", "Unique_Words"])

# Display the result
print(decade_df)

   Decade                                       Unique_Words
0    2010  [ensemble, coat, dress, tote, sculpture, print...
1    2020  [ensemble, boots, dress, pot, shoes, slippers,...


In [53]:
decade_df['Word_Count'] = decade_df['Unique_Words'].apply(len)

print(decade_df)

   Decade                                       Unique_Words  Word_Count
0    2010  [ensemble, coat, dress, tote, sculpture, print...          12
1    2020  [ensemble, boots, dress, pot, shoes, slippers,...           9


In [ ]:
import pandas as pd
from sqlalchemy import create_engine  # Import create_engine
import re
from collections import Counter

# Create the engine for the SQLite database
engine = create_engine('sqlite:///../data/processed/social_movements.db')  

# Read data from the SmithsonianArtworks table
df = pd.read_sql_query("SELECT * FROM SmithsonianArtworks;", engine)

# Load the dataset
filepath = '../data/processed/american_artworks_with_dates.csv'
df = pd.read_csv(filepath)

# Convert 'ProductionYear' to numeric, handling errors
df["ProductionYear"] = pd.to_numeric(df["ProductionYear"], errors='coerce')

# Filter the dataset for years between 1920 and 2010
df = df[(df["ProductionYear"] >= 1920) & (df["ProductionYear"] <= 2010)]

# Define common filler words to exclude
stopwords = set(["and", "or", "for", "the", "of", "in", "on", "with", "to", "a", "an", "by", "at", "as", "from", "untitled", "no", "1", "untitled"])

# Function to clean and extract words from a title (filtering stopwords and numbers)
def process_title(title):
    words = re.findall(r'\b\w+\b', title.lower())  # Extract words
    return [word for word in words if word not in stopwords and not word.isdigit()]  

# Apply title processing function to the 'Title' column
df["ProcessedTitles"] = df["Title"].dropna().apply(process_title)

# Group titles by decade
df["Decade"] = (df["ProductionYear"] // 10) * 10  # Convert year to decade

# Function to get unique words for each group
def get_unique_words(group):
    all_words = [word for title in group for word in title]  # Flatten the list of words
    word_counts = Counter(all_words)  # Count word occurrences
    return list(word_counts.keys())  # Get unique words

# Group by decade and apply the word counting function
decade_word_counts = df.groupby("Decade")["ProcessedTitles"].apply(lambda x: get_unique_words(x)).reset_index()

# Rename columns
decade_word_counts.columns = ["Decade", "Unique_Words"]

print(decade_word_counts)


   Decade                                       Unique_Words
0  1920.0  [standing, articulated, figure, kenyon, cox, f...
1  1930.0  [shapes, fear, street, shrine, teasel, belliss...
2  1940.0  [field, seated, nude, girl, sister, ella, frog...
3  1950.0  [tangiers, new, england, hostess, madonna, sol...
4  1960.0  [man, three, donkeys, children, swimming, moth...
5  1970.0  [tieback, one, pair, landscape, mattress, make...
6  1980.0  [lady, samuel, mink, two, gentlemen, comb, you...
7  1990.0  [memory, vessel, doll, parts, whaler, s, quilt...
8  2000.0  [timelines, tenements, chatelaine, boy, fancy,...
9  2010.0  [cross, catalogue, number, series, accession, ...
